In [1]:
from IPython.display import display
import os
import shutil





In [5]:
import numpy as np
import cv2
import glob
import numpy as np
import keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [60]:
IMAGE_SIZE = 400
BATCH_SIZE = 8
CHANNELS = 3
EPOCHS = 1

In [61]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "./initial training set", 
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 154 files belonging to 2 classes.


In [62]:
class_names = dataset.class_names
class_names

['mosquito bites on skin', 'tick bites on skin']

In [63]:
len(dataset)

20

In [64]:
train_size = 0.8
train_set_length = len(dataset) * train_size
train_set_length

16.0

In [65]:
train_ds= dataset.take(16)

In [66]:
len(train_ds)

16

In [67]:
test_ds = dataset.skip(16) 
len(test_ds)

4

In [68]:
val_size = 0.1
len(dataset)*val_size

2.0

In [69]:
val_ds = test_ds.take(2)

In [70]:
len(val_ds)

2

In [71]:
test_ds = test_ds.skip(2)
len(test_ds)

2

In [72]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split= 0.1, test_split = 0.1, shuffle=True, shuffle_size =10000):
    ds_size = len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [73]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [74]:
len(train_ds)
len(val_ds)
len(test_ds)

2

In [75]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [76]:
from tensorflow.keras import models, layers

In [77]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [78]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2) 
])

In [79]:
Input_shape=(BATCH_SIZE, IMAGE_SIZE,IMAGE_SIZE, CHANNELS)
n_classes = 2
model = models.Sequential([
resize_and_rescale, 
                          data_augmentation,
                          layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE,IMAGE_SIZE)),
                          layers.MaxPooling2D((2,2)),
                          ##########################
                          layers.Conv2D(64, kernel_size = (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, kernel_size = (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Conv2D(64, (3,3), activation='relu'),              
                          layers.MaxPooling2D((2,2)),
                          layers.Flatten(),
                          layers.Dense(64, activation='relu'),
                          layers.Dense(n_classes, activation='softmax')
])
model.build(input_shape=Input_shape)

In [80]:
model.summary()
                        

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (8, 400, 400, 3)          0         
                                                                 
 sequential_6 (Sequential)   (None, 400, 400, 3)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 398, 398, 32)      896       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 199, 199, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 197, 197, 64)      18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 98, 98, 64)       0         
 g2D)                                                 

In [81]:
model.compile(
    optimizer ='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
)

In [82]:
history = model.fit(
    train_ds,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
    
)

InvalidArgumentError: Graph execution error:

Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_12214]

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

trainImages = np.asarray(trainImages)
trainLabels = np.asarray(trainLabels).astype('float32')
validationLabels = np.asarray(validationLabels).astype('float32')
validationImages = np.asarray(validationImages)

history = model.fit(trainImages, trainLabels, epochs=10, 
                    validation_data=(validationImages, validationLabels))

In [ ]:
trainImages.shape()
trainLabels.shape()

validationImages.shape()
validationLabels.shape()

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
